<a href="https://colab.research.google.com/github/shivamchoudhury06/ML_Projects/blob/master/Content_Based_Filter_20240909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import nltk
from nltk.stem import PorterStemmer

#from fuzzywuzzy import fuzz

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
movie = pd.read_csv("/content/drive/My Drive/home-credit-default-risk/movies_metadata.csv", low_memory=False)

In [ ]:
movie.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.00,81.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.70,5415.00
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.00,104.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.90,2413.00


In [3]:
# Columns to Include
#genres, budget, original_language, original_title, overview, production_companies, production_countries, spoken_languages, title

movie_list = movie[[ 'genres', 'id', 'original_language', 'original_title', 'overview', 'production_companies', 'production_countries']]
top_1000_movies = movie_list[:1000]

# Cleaning the data


#### Finding missing values


In [ ]:
# Function to calculate missing values by column# Funct
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values_table(top_1000_movies)

Your selected dataframe has 9 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [4]:
top_1000_movies.dropna(inplace = True)

<ipython-input-4-bccfdf3b1a15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies.dropna(inplace = True)


### Extracting required values from the dictionary

In [ ]:
print(top_1000_movies['genres' ][0])
print(top_1000_movies['production_companies' ][0])
print(top_1000_movies['production_countries' ][0])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]


In [5]:
import ast

# This fuction will extract the required values from the dictionary and append in a list
def extract_data(text):
    List = []
    if not text:  # Check if text is empty or None
        return List

    try:
        for i in ast.literal_eval(text): #inorder to convert the string to list we use ast.literal

            if 'name' in i and i['name']:  # Check if 'name' exists and is not empty
                #print(i['name'])
                List.append(i['name'])
    except (ValueError, SyntaxError):
        # Handle cases where text cannot be parsed as a list/dict
        return List

    return List


# Applying extract_data() to required columns
top_1000_movies['production_companies'] = top_1000_movies['production_companies'].apply(extract_data)
top_1000_movies['production_countries'] = top_1000_movies['production_countries'].apply(extract_data)
top_1000_movies['genres'] = top_1000_movies['genres'].apply(extract_data)


<ipython-input-5-61024d049ef8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies['production_companies'] = top_1000_movies['production_companies'].apply(extract_data)
<ipython-input-5-61024d049ef8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies['production_countries'] = top_1000_movies['production_countries'].apply(extract_data)
<ipython-input-5-61024d049ef8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [ ]:
# Check for Genre, production_companies, production_companies. Data should be stored in list
top_1000_movies.head(3)

,genres,id,original_language,original_title,overview,production_companies,production_countries
0,"[Animation, Comedy, Family]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[United States of America]
1,"[Adventure, Fantasy, Family]",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America]
2,"[Romance, Comedy]",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Warner Bros., Lancaster Gate]",[United States of America]


### Cleaning textual data

In [ ]:
# handle overview (converting to list)

top_1000_movies.iloc[0]['overview']

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [6]:
# Extract Overview column and splitting the column to cancatenate it in a single list
def overview_extract(column):
  # Convert the 'overview' column to string type to handle potential float values.
  # Use a lambda function to split the string only if it's not 'nan' (representing missing values).
  # If it's 'nan', replace it with an empty list.

  column = column.astype(str).apply(lambda x: x.split() if x != 'nan' else [])
  return column

top_1000_movies['overview'] = overview_extract(top_1000_movies['overview'])


<ipython-input-6-70bc79a427ad>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies['overview'] = overview_extract(top_1000_movies['overview'])


In [ ]:
# Overview column should be in list
top_1000_movies.head(3)

,genres,id,original_language,original_title,overview,production_companies,production_countries
0,"[Animation, Comedy, Family]",862,en,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...",[Pixar Animation Studios],[United States of America]
1,"[Adventure, Fantasy, Family]",8844,en,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America]
2,"[Romance, Comedy]",15602,en,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Warner Bros., Lancaster Gate]",[United States of America]


### Applying Stem function

In [7]:
# Stemming
# playing -> play, plays-> play
ps = PorterStemmer()
def stems(text):
    # Check if the input is a list
    if isinstance(text, list):
        # Apply stemming to each element in the list
        return [" ".join([ps.stem(word) for word in sentence.split()]) for sentence in text]
    else:
        # Apply stemming to the single string
        return " ".join([ps.stem(word) for word in text.split()])


In [8]:
# Applying stemming function on the overview column
top_1000_movies['overview'] = top_1000_movies['overview'].apply(stems)

<ipython-input-8-3d1a010359bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies['overview'] = top_1000_movies['overview'].apply(stems)


### Creating a master column and storing it in dictionary

In [9]:
def create_textual_representation(row):
  textual_representation = f"""
  Title: {row['original_title']},
  Genre: {row['genres']},
  Language: {row['original_language']},
  Overview: {row['overview']},
  Production Companies: {row['production_companies']},
  Production Countries: {row['production_countries']},
  """

  return textual_representation

top_1000_movies["master_column"] = top_1000_movies.apply(create_textual_representation, axis=1)

<ipython-input-9-783feacbc54f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies["master_column"] = top_1000_movies.apply(create_textual_representation, axis=1)


In [ ]:
print(top_1000_movies['master_column'][0])


  Title: Toy Story,
  Genre: ['Animation', 'Comedy', 'Family'],
  Language: en,
  Overview: ['led', 'by', 'woody,', "andy'", 'toy', 'live', 'happili', 'in', 'hi', 'room', 'until', "andy'", 'birthday', 'bring', 'buzz', 'lightyear', 'onto', 'the', 'scene.', 'afraid', 'of', 'lose', 'hi', 'place', 'in', "andy'", 'heart,', 'woodi', 'plot', 'against', 'buzz.', 'but', 'when', 'circumst', 'separ', 'buzz', 'and', 'woodi', 'from', 'their', 'owner,', 'the', 'duo', 'eventu', 'learn', 'to', 'put', 'asid', 'their', 'differences.'],
  Production Companies: ['Pixar Animation Studios'],
  Production Countries: ['United States of America'],
  


### Concatenating
Concatenating the required columns and applying TFIDF vectorizer on the concatenated column to find out the recommended movies

In [10]:
# Concatinate all and make a concatenate column [FOR TRIAL PURPOSE]
# NOTE: We need to add original_language column by appending it in a list
top_1000_movies['concat_column'] = top_1000_movies['genres']+top_1000_movies['production_companies']+top_1000_movies['production_countries']+top_1000_movies['overview']


<ipython-input-10-3863181a76e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies['concat_column'] = top_1000_movies['genres']+top_1000_movies['production_companies']+top_1000_movies['production_countries']+top_1000_movies['overview']


In [11]:
print(top_1000_movies['concat_column'][0])

['Animation', 'Comedy', 'Family', 'Pixar Animation Studios', 'United States of America', 'led', 'by', 'woody,', "andy'", 'toy', 'live', 'happili', 'in', 'hi', 'room', 'until', "andy'", 'birthday', 'bring', 'buzz', 'lightyear', 'onto', 'the', 'scene.', 'afraid', 'of', 'lose', 'hi', 'place', 'in', "andy'", 'heart,', 'woodi', 'plot', 'against', 'buzz.', 'but', 'when', 'circumst', 'separ', 'buzz', 'and', 'woodi', 'from', 'their', 'owner,', 'the', 'duo', 'eventu', 'learn', 'to', 'put', 'asid', 'their', 'differences.']


In [ ]:
top_1000_movies.columns

Index(['genres', 'id', 'original_language', 'original_title', 'overview',
       'production_companies', 'production_countries', 'master_column',
       'concat_column'],
      dtype='object')

In [12]:
# droping those extra columns
top_1000_movies_master_col = top_1000_movies[['id','original_title','master_column']]
top_1000_movies_concat_col = top_1000_movies[['id','original_title','concat_column']]

In [13]:
top_1000_movies_concat_col.head()

,id,original_title,concat_column
0,862,Toy Story,"[Animation, Comedy, Family, Pixar Animation St..."
1,8844,Jumanji,"[Adventure, Fantasy, Family, TriStar Pictures,..."
2,15602,Grumpier Old Men,"[Romance, Comedy, Warner Bros., Lancaster Gate..."
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance, Twentieth Century Fox..."
4,11862,Father of the Bride Part II,"[Comedy, Sandollar Productions, Touchstone Pic..."


In [14]:
top_1000_movies_master_col.head(3)

,id,original_title,master_column
0,862,Toy Story,"\n Title: Toy Story,\n Genre: ['Animation', ..."
1,8844,Jumanji,"\n Title: Jumanji,\n Genre: ['Adventure', 'F..."
2,15602,Grumpier Old Men,"\n Title: Grumpier Old Men,\n Genre: ['Roman..."


In [15]:
# Stemming
# playing -> play, plays-> play
ps = PorterStemmer()
def stems(text):
    # Check if the input is a list
    if isinstance(text, list):
        # Apply stemming to each element in the list
        return [" ".join([ps.stem(word) for word in sentence.split()]) for sentence in text]
    else:
        # Apply stemming to the single string
        return " ".join([ps.stem(word) for word in text.split()])


In [16]:
# Applying stemming function to the master column
top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(stems)

<ipython-input-16-9358d30dc524>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(stems)


In [17]:
top_1000_movies_concat_col['concat_column'][0][:10]

['anim',
 'comedi',
 'famili',
 'pixar anim studio',
 'unit state of america',
 'led',
 'by',
 'woody,',
 "andy'",
 'toy']

In [18]:
print(top_1000_movies_master_col['master_column'][0])


  Title: Toy Story,
  Genre: ['Animation', 'Comedy', 'Family'],
  Language: en,
  Overview: ['led', 'by', 'woody,', "andy'", 'toy', 'live', 'happili', 'in', 'hi', 'room', 'until', "andy'", 'birthday', 'bring', 'buzz', 'lightyear', 'onto', 'the', 'scene.', 'afraid', 'of', 'lose', 'hi', 'place', 'in', "andy'", 'heart,', 'woodi', 'plot', 'against', 'buzz.', 'but', 'when', 'circumst', 'separ', 'buzz', 'and', 'woodi', 'from', 'their', 'owner,', 'the', 'duo', 'eventu', 'learn', 'to', 'put', 'asid', 'their', 'differences.'],
  Production Companies: ['Pixar Animation Studios'],
  Production Countries: ['United States of America'],
  


In [19]:
# Converting list to str
def convert_list_to_string(lst):
    return ' '.join(lst)

def convert_to_lower_case(col):
  return col.lower()

## Applying convert_list_to_string and convert_to_lower_case to concat column for stemming purpose or TF-IDF Vectorizer
top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(convert_list_to_string)
top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(convert_to_lower_case)


# new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
# new_df.head()

<ipython-input-19-6ebf116d052b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(convert_list_to_string)
<ipython-input-19-6ebf116d052b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1000_movies_concat_col['concat_column'] = top_1000_movies_concat_col['concat_column'].apply(convert_to_lower_case)


### Vectorization using TF-IDF

In [23]:
def tf_idf_vectorizer(column):
  #TF-IDF Vectorization
  vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
  matrix = vectorizer.fit_transform(column)
  return matrix

In [24]:
top_1000_movies_concat_col_matrix = tf_idf_vectorizer(top_1000_movies_concat_col["concat_column"])
top_1000_movies_master_col_matrix = tf_idf_vectorizer(top_1000_movies_master_col["master_column"])

In [25]:
print(top_1000_movies_concat_col_matrix.shape)
print(top_1000_movies_master_col_matrix.shape)

(988, 1000)
(988, 1000)


### Cosine similarity

In [26]:
def content_similarity(matrix):
  cosine_similarities = linear_kernel(matrix)
  return cosine_similarities

In [27]:
top_1000_movies_concat_col_matrix_similarity = content_similarity(top_1000_movies_concat_col_matrix)
top_1000_movies_master_col_matrix_similarity = content_similarity(top_1000_movies_master_col_matrix)

### Recommending movies

In [28]:
def recommend(movie, dataframe, similarity_matrix):
    # Find the index of the movie in the dataframe
    index = dataframe[dataframe['original_title'] == movie].index[0]

    # Compute the list of distances (similarity scores) from the movie to all others
    distances = sorted(list(enumerate(similarity_matrix[index])), reverse=True, key=lambda x: x[1])

    # Print the top 5 similar movies with their similarity scores rounded to 3 decimal places
    print(f"**Top 5 recommendations for:'{movie}':**")
    for i in distances[1:6]:  # Skip the first one since it's the movie itself
        print(dataframe.iloc[i[0]].original_title,  round(i[1], 3))


In [31]:
recommend('Jumanji',top_1000_movies_concat_col,top_1000_movies_concat_col_matrix_similarity)

**Top 5 recommendations for:'Jumanji':**
Sudden Death 0.248
Maverick 0.22
Mortal Kombat 0.207
D3: The Mighty Ducks 0.206
Jury Duty 0.205


In [30]:
recommend('Jumanji',top_1000_movies_master_col,top_1000_movies_master_col_matrix_similarity)

**Top 5 recommendations for:'Jumanji':**
Sudden Death 0.233
Kazaam 0.219
D3: The Mighty Ducks 0.219
Maverick 0.213
Picnic 0.213


## BERT Vectorization

In [33]:

pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.3 MB/s eta 0:00:00


In [34]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [35]:
bert = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
# Convert the Pandas Series to a list of strings.
sentence_embeddings = bert.encode(top_1000_movies_concat_col['concat_column'].tolist())

In [38]:
sim_matrix_bert_model = content_similarity(sentence_embeddings)


In [40]:
recommend('Jumanji',top_1000_movies_concat_col,sim_matrix_bert_model)

**Top 5 recommendations for:'Jumanji':**
The Nightmare Before Christmas 211.251
Super Mario Bros. 208.617
The Aristocats 204.909
Coldblooded 203.288
Fresh 202.775


In [ ]:
index = dataframe[dataframe['original_title'] == movie].index[0]

In [ ]:
import ast

def convert(text):
    List = []
    if not text:  # Check if text is empty or None
        return List

    try:
        for i in ast.literal_eval(text): #inorder to convert the string to list we use ast.literal

            if 'name' in i and i['name']:  # Check if 'name' exists and is not empty
                #print(i['name'])
                List.append(i['name'])
    except (ValueError, SyntaxError):
        # Handle cases where text cannot be parsed as a list/dict
        return List

    return List


In [ ]:
movie.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
# Columns to Include
#genres, budget, original_language, original_title, overview, production_companies, production_countries, spoken_languages, title

movie_list = movie[['budget', 'genres', 'id', 'original_language', 'original_title', 'overview', 'production_companies', 'production_countries']]
movie_list_top_1000 = movie_list[:1000]

In [ ]:
movie_list_top_1000.head(3)

,budget,genres,id,original_language,original_title,overview,production_companies,production_countries
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o..."
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o..."
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o..."


In [ ]:
# Function to calculate missing values by column# Funct
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()

        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)

        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(movie_list_top_1000)

Your selected dataframe has 8 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
overview,12,1.20


In [ ]:
movie_list_top_1000.dropna(inplace=True)

<ipython-input-33-070aa30e76e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_list_top_1000.dropna(inplace=True)


In [ ]:
movie_list_top_1000['production_companies'] = movie_list_top_1000['production_companies'].apply(convert)
movie_list_top_1000['production_countries'] = movie_list_top_1000['production_countries'].apply(convert)
movie_list_top_1000['genres'] = movie_list_top_1000['genres'].apply(convert)


<ipython-input-34-6061dd3eb166>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_list_top_1000['production_companies'] = movie_list_top_1000['production_companies'].apply(convert)
<ipython-input-34-6061dd3eb166>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_list_top_1000['production_countries'] = movie_list_top_1000['production_countries'].apply(convert)
<ipython-input-34-6061dd3eb166>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [ ]:
movie_list_top_1000.head()

,budget,genres,id,original_language,original_title,overview,production_companies,production_countries
0,30000000,"[Animation, Comedy, Family]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[United States of America]
1,65000000,"[Adventure, Fantasy, Family]",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America]
2,0,"[Romance, Comedy]",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Warner Bros., Lancaster Gate]",[United States of America]
3,16000000,"[Comedy, Drama, Romance]",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",[Twentieth Century Fox Film Corporation],[United States of America]
4,0,[Comedy],11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[Sandollar Productions, Touchstone Pictures]",[United States of America]


In [ ]:
movie_list[movie_list['id'] == '862'].head()

,budget,genres,id,original_language,original_title,overview,production_companies,production_countries
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o..."


In [ ]:
def create_textual_representation(row):
  textual_representation = f"""
  Title: {row['original_title']},
  Budget: {row['budget']},
  Language: {row['original_language']},
  Overview: {row['overview']},
  Production Companies: {row['production_companies']},
  Production Countries: {row['production_countries']},
  """
  return textual_representation

In [ ]:
movie_list_top_1000["master_column"] = movie_list.apply(create_textual_representation, axis=1)

<ipython-input-37-1bbcf78bdac3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_list_top_1000["master_column"] = movie_list.apply(create_textual_representation, axis=1)


In [ ]:
movie_list_top_1000["master_column"][0]

"\n  Title: Toy Story,\n  Budget: 30000000,\n  Language: en,\n  Overview: Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.,\n  Production Companies: [{'name': 'Pixar Animation Studios', 'id': 3}],\n  Production Countries: [{'iso_3166_1': 'US', 'name': 'United States of America'}],\n  "

In [ ]:
def tf_idf_vectorizer(column):
  #if-idf Vectorization
  vectorizer = TfidfVectorizer()
  matrix = vectorizer.fit_transform(column)
  return matrix

In [ ]:
movie_list_top_1000["master_column"] = movie_list_top_1000["master_column"].fillna("")

<ipython-input-17-359ebb78126f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_list_top_1000["master_column"] = movie_list_top_1000["master_column"].fillna("")


In [ ]:
matrix = tf_idf_vectorizer(movie_list_top_1000["master_column"])

In [ ]:
matrix.shape

(1000, 11939)

In [ ]:
def content_similarity():
  cosine_similarities = linear_kernel(matrix,matrix)
  return cosine_similarities